In [ ]:
!python -m pip install -qq kaggle

In [ ]:
%%bash
mkdir ~/.kaggle
cp kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json
kaggle datasets download andrewmvd/retinal-disease-classification
unzip -qq /content/retinal-disease-classification.zip
mv "Training_Set/Training_Set/Training" "Training"
mv "Training" "train"
mv "Training_Set/Training_Set/RFMiD_Training_Labels.csv" "RFMiD_Training_Labels.csv"
mv "RFMiD_Training_Labels.csv" "rfmid_training_labels.csv"
mv "Evaluation_Set/Evaluation_Set/Validation" "Validation"
mv "Validation" "valid"
mv "Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv" "RFMiD_Validation_Labels.csv"
mv "RFMiD_Validation_Labels.csv" "rfmid_validation_labels.csv"
mv "Test_Set/Test_Set/Test" "Test"
mv "Test" "test"
mv "Test_Set/Test_Set/RFMiD_Testing_Labels.csv" "RFMiD_Testing_Labels.csv"
mv "RFMiD_Testing_Labels.csv" "rfmid_testing_labels.csv"
rm -rf "Evaluation_Set" "Test_Set" "Training_Set"

In [ ]:
import os                        # Module for interacting with the operating system
import string                    # Module with utility functions for working with strings
import numpy as np               # Numerical computing library
import pandas as pd              # Library for data manipulation and analysis
import seaborn as sns            # Data visualization library built on top of matplotlib
import tensorflow as tf         # Deep learning framework
from matplotlib import pyplot as plt  # Plotting library
from PIL import Image            # Library for working with images
import re                        # Module for working with regular expressions

In [ ]:
# read all csv files
train_metadata=pd.read_csv('rfmid_training_labels.csv')                         # read training data csv (metadata)
valid_metadata=pd.read_csv('rfmid_validation_labels.csv')                       # read training data csv (metadata)
test_metadata=pd.read_csv('rfmid_testing_labels.csv')                           # read training data csv (metadata)
# add columns for type of data dataframe
train_metadata['setType']='train'
valid_metadata['setType']='valid'
test_metadata['setType']='test'
# merage all to one dataset
metadata=pd.concat([train_metadata,valid_metadata,test_metadata],ignore_index=True)# concatinate all
metadata.head()                                                                 # metadata head

In [ ]:
# get all images addresses
images_addresses=dict(train=list(),valid=list(),test=list())                    # make dictionary for dataset
for key,value in images_addresses.items():                                      # get key and value
  for imageType in ['jpg','jpeg','png','bmp','gif']:                            # search for each image type
    images_addresses[key].extend(glob(key+r'/*'+imageType))                     # add each type image address
  print(f'Total images found in {key} is {len(images_addresses[key])}')         # print total images count

In [ ]:
#make plot data
for images,labels in train.take(1):break
images_to_plot:int=9
plt.figure(figsize=(15,15))
for index in range(9):
  plt.subplot(3,3,index+1)
  plt.imshow(images[index].numpy().astype('uint8'))
  plt.axis('off')
  plt.title(training_classes[labels[index].numpy()])
plt.tight_layout()
plt.show()

In [ ]:
# laod base model - MobileNetV2
base_model=tf.keras.applications.MobileNetV2(include_top=False,weights="imagenet",input_shape=(160,160,3))
#base_model.summary(line_length=120,positions=None,print_fn=None,expand_nested=True,show_trainable=False,layer_range=None)# print base model summary
base_model.trainable=False                                                      # make freeze base

In [ ]:
inputs=tf.keras.layers.Input(shape=(160,160,3),batch_size=None,name='model_inputs')# add input layer
model_x=tf.keras.layers.Rescaling(1/255,offset=1.0,name='rescaler')(inputs)     # rescale inputs
model_x=base_model(model_x)                                                     # feed inputs to base model
model_x=tf.keras.layers.Flatten(name='flatten')(model_x)                         # make flatten outputs
model_x=tf.keras.layers.Dense(1024,activation='relu')(model_x)                  # apply dense layer
outputs=tf.keras.layers.Dense(len(training_classes),activation='softmax')(model_x)# output layer
model=tf.keras.Model(inputs=[inputs],outputs=[outputs],name='Xception')         # build model
model.summary(line_length=120,positions=None,print_fn=None,expand_nested=True,show_trainable=True,layer_range=None)# print model summary

In [ ]:
# make compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=tf.keras.metrics.SparseCategoricalAccuracy())

In [ ]:
# make early stopping
early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy',min_delta=0,patience=10,verbose=1,mode='auto',restore_best_weights=True)

# make train model
history=model.fit(train,batch_size=None,epochs=10_000,verbose=2,callbacks=[early_stop],
                  validation_split=0,validation_data=valid,shuffle=True)

In [ ]:
# see model history (first five epoches)
pd.DataFrame(history.history).head()

In [ ]:
# see model history (last five epoches)
pd.DataFrame(history.history).tail()

In [ ]:
# make plot training accuracy and validation accuracy
pd.DataFrame(history.history)[['sparse_categorical_accuracy','val_sparse_categorical_accuracy']].plot(marker='.')# make plot
plt.grid(),plt.title('Training Accuracy'),plt.xlabel('epochs'),plt.ylabel('accuracy')# add grid, title and axis labels
plt.show()

In [ ]:
# make plot training losses and validation losses
pd.DataFrame(history.history)[['loss','val_loss']].plot(marker='.')             # make plot
plt.grid(),plt.title('Training Losses'),plt.xlabel('epochs'),plt.ylabel('loss') # add grid, title and axis labels
plt.show()                                                                      # make plot show

In [ ]:
# evaluate model
results=model.evaluate(test,batch_size=None,verbose="auto",sample_weight=None,return_dict=True)
print('Testing results:',results)

In [ ]:
# make save model (weights and configuration)
model.save('traffic_sign_classifier',overwrite=True,save_format=None)           # make save

In [ ]:
!zip -r -qq "traffic_sign_classifier.zip" "traffic_sign_classifier"             # make zip model
# make auto download model weights
from google.colab import files                                                  # load file class
files.download('traffic_sign_classifier.zip')                                   # download model zip file